<a href="https://colab.research.google.com/github/aronyo24/research/blob/main/Digitalization_of_Land_Record_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.patches as mpatches


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Research Paper/digi_land_re.csv")
df

In [ ]:
df.columns.tolist()

In [ ]:
df=df.drop(columns=['Recommendation from elites','Device Used for submitting application','Unnamed: 22', 'Unnamed: 23','Unnamed: 24'])
#df

In [ ]:
df.columns.tolist()

In [ ]:
df.rename(columns={
    "Service Type ":"service_type",
    'Location':'location',
    "Education": "education",
    "Phone User": "phone_type",
    "Laptop Access": "laptop_access",
    "Internet Access": "internet_access",
    "Who Submitted Application" :"who_submitted_application",
    "Device Used for submitting application": "device_used",
    "Application Submission times": "app_submission_times",
    "Cost of Submitting Application ": "app_cost",
    "Cost for e-mutation": "e_mutation_cost",
    "To whom, pay": "payment_receiver",
    "Days to get the service": "service_days",
    "Citizen Awareness of Circulars": "awareness_circulars",
    "Citizen's Awareness for Complaints": "awareness_complaints",
    "Visits to government offices": "gov_office_visits",
    "Cash transaction to govt Offices": "cash_transaction",
    "Citizen's Satistaction": "citizen_satisfaction",
    "Recommendation from elites":"recommendation_f_elites",
    "Previous experience of mutation":"Previous_experience_mut"

}, inplace=True)

df


In [ ]:
##  Education: 1 = Secondary, 0 = Uneducated ,2= Highly Educated
df['education'] = df['education'].apply(lambda x: 0 if x in ["ill", "no education"] else (1 if x in ["eight", "ssc"] else 2))



##  Gender: 1 = , 0 = Female
df['Gender'] = df['Gender'].apply(lambda x: 0 if x == "2" else 1)

##  Phone Type: 1 = Android, 0 = Button Phone
df['phone_type'] = df['phone_type'].apply(lambda x: 1 if x == 1 else 0)

##  Laptop Access: 1 = Yes, 0 = No
df['laptop_access'] = df['laptop_access'].apply(lambda x: 1 if x == 1 else 0)

## Internet Access: 1 = Yes, 0 = No
df['internet_access'] = df['internet_access'].apply(lambda x: 1 if x == 1 else 0)

## Citizen Awareness of Circulars: 1 = Yes, 0 = No
df['awareness_circulars'] = df['awareness_circulars'].apply(lambda x: 1 if x == 1 else 0)

## Citizen's Awareness for Complaints: 1 = Yes, 0 = No
df['awareness_complaints'] = df['awareness_complaints'].apply(lambda x: 1 if x == 1 else 0)
## Device Used for submitting application 1 = laptop ,0 = phone
#df['device_used'] = df['device_used'].apply(lambda x: 1 if x == 1 else 0)

## Cash transaction to govt Offices 1 = yes ,0 = no
df['cash_transaction'] = df['cash_transaction'].apply(lambda x: 1 if x == 1 else 0)

##Citizen's Satistaction 1= yes , 0 = no
df['citizen_satisfaction'] = df['citizen_satisfaction'].apply(lambda x: 1 if x == 1 else 0)



In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df = df.dropna()

df


In [ ]:
df.hist(figsize=(15, 13), bins=20)
plt.show()

In [ ]:
df[['app_cost', 'e_mutation_cost','service_days']]
# in column e mutation cost means money range 1= 1170, 2 = 1170 to 2000 , 3= 2001 to 3000 & 4= 3001 to 4000
# in column service days means m range 1= 28 ,2 = 28 to 45

app_cost_map = {
    1: 100,
    2: 150,
    3: 200
}

# Define the mappings
e_mutation_cost_map = {
    1: 1170,
    2: 1585,
    3: 2500,
    4: 3500
}

service_days_map = {
    1: 28,
    2: 36
}


# Map values for e_mutation_cost if they are in [1, 2, 3, 4]
df['app_cost'] = df['app_cost'].apply(lambda x: app_cost_map.get(x, x))
df['e_mutation_cost'] = df['e_mutation_cost'].apply(lambda x: e_mutation_cost_map.get(x, x))

# Map values for service_days if they are in [1, 2]
df['service_days'] = df['service_days'].apply(lambda x: service_days_map.get(x, x))

df


In [ ]:
df.to_csv('final_digi.csv', index=False)

After processing data from both digital and manual systems.**bold text**

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Research Paper/processed_data_dig_man.csv")
df

In [ ]:
df.hist(figsize=(15, 13), bins=20)
plt.show()

In [ ]:
df.columns

In [ ]:
sns.histplot(x=df['service_type'],y=df['service_days'], data=df)


In [ ]:

df['Treatment'] = df['service_type']

#  Post-digitalization period -------- service_days
df['Post'] = (df['service_days'] <=39 ).astype(int)

# DID Interaction Term
df['Treatment_Post'] = df['Treatment'] * df['Post']

In [ ]:
model = smf.ols(
    "service_days ~ Treatment + Post + Treatment_Post + Age + Gender + education + phone_type + "
    "laptop_access + internet_access + who_submitted_application + app_submission_times + app_cost + "
    "e_mutation_cost + payment_receiver + awareness_circulars + awareness_complaints + gov_office_visits + "
    "cash_transaction + citizen_satisfaction",
    data=df
).fit()


print(model.summary())

In [ ]:


coefs = model.params
conf_int = model.conf_int()
conf_int.columns = ['Lower Bound', 'Upper Bound']


coef_df = pd.DataFrame({'Coefficient': coefs, 'Lower Bound': conf_int['Lower Bound'], 'Upper Bound': conf_int['Upper Bound']})
coef_df = coef_df.drop('Intercept')  #
coef_df = coef_df.sort_values(by="Coefficient", ascending=False)

#positive = red, negative = blue
colors = ['red' if coef > 0 else 'blue' for coef in coef_df['Coefficient']]


plt.figure(figsize=(10, 6))
bars = plt.barh(coef_df.index, coef_df['Coefficient'], color=colors, xerr=[coef_df['Coefficient'] - coef_df['Lower Bound'], coef_df['Upper Bound'] - coef_df['Coefficient']])
plt.xlabel("Effect on Service Days")
plt.title("Impact of Different Factors on Service Time")
plt.axvline(x=0, color="black", linestyle="dashed")
plt.gca().invert_yaxis()


for bar, coef in zip(bars, coef_df['Coefficient']):
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f"{coef:.2f}", ha='left' if coef > 0 else 'right', va='center', fontsize=10)

plt.show()


In [ ]:


did_plot = df.groupby(['Treatment', 'Post'])['service_days'].mean().reset_index()


did_plot['System'] = did_plot['Treatment'].map({0: 'Manual', 1: 'Digital'})
did_plot['Period'] = did_plot['Post'].map({0: 'Before', 1: 'After'})


did_pivot = did_plot.pivot(index='Period', columns='System', values='service_days')


plt.figure(figsize=(8, 5))
sns.lineplot(data=did_pivot, markers=True, dashes=False)
plt.title("Effect of Digitalization on Service Time (DID Analysis)")
plt.ylabel("Average Service Days")
plt.xlabel("Time Period")
plt.axvline(0.5, color="black", linestyle="dashed", label="Digitalization Begins")
plt.legend(title="Service Type")
plt.show()


In [ ]:
sns.countplot(x=df["location"], hue=df["Treatment"])
plt.title("Urban vs. Rural")
plt.xlabel("Location")
plt.ylabel("Number of Users")
plt.legend(title="Service Type (0 = Manual, 1 = Digital)")
plt.show()

In [ ]:

education_labels = {0: "Uneducated", 1: "Secondary", 2: "Highly Educated"}

df["Education Label"] = df["education"].map(education_labels)


df_counts = df.groupby(["Education Label", "Treatment"]).size().reset_index(name="Count")


plt.figure(figsize=(8, 5))
ax = sns.barplot(
    x="Education Label",
    y="Count",
    hue="Treatment",
    data=df_counts,
    palette={0: "green", 1: "orange"},
    dodge=True,
    capsize=0.1
)


plt.title("Number of People Using Digital vs. Manual Systems by Education Level")
plt.xlabel("Education Level")
plt.ylabel("Number of People")

# Fix legend colors
manual_patch = mpatches.Patch(color="green", label="Manual System")
digital_patch = mpatches.Patch(color="orange", label="Digital System")
plt.legend(handles=[manual_patch, digital_patch], title="Service Type", fontsize=11, loc="best")



In [ ]:

plt.figure(figsize=(8, 5))
ax = sns.barplot(
    x="Treatment_Post",
    y="citizen_satisfaction",
    data=df,
    hue="Treatment_Post",
    palette={0: "blue", 1: "orange"},
    capsize=0.1,
    dodge=False
)


plt.title("Impact of Digitalization on Citizen Satisfaction")
plt.xlabel("Digitalization Impact")
plt.ylabel("Average Citizen Satisfaction")


ax.set_xticklabels(["Before Digitalization", "After Digitalization"])


handles = [
    plt.Rectangle((0,0),1,1, color="blue", alpha=0.6, label="Before Digitalization"),
    plt.Rectangle((0,0),1,1, color="orange", alpha=0.6, label="After Digitalization")
]
plt.legend(handles=handles, title="Digitalization", fontsize=11)




In [ ]:

df_analysis = df.groupby(["internet_access", "Treatment_Post"])["app_submission_times"].mean().reset_index()

df_analysis["internet_access_label"] = df_analysis["internet_access"].map({0: "No Internet", 1: "Has Internet"})
df_analysis["digitalization_label"] = df_analysis["Treatment_Post"].map({0: "Before Digitalization", 1: "After Digitalization"})


plt.figure(figsize=(8, 5))
ax = sns.barplot(
    x="internet_access_label",
    y="app_submission_times",
    hue="digitalization_label",
    data=df_analysis,
    palette={"Before Digitalization": "blue", "After Digitalization": "orange"},
    capsize=0.1
)


plt.title("Effect of Internet Access on Application Submission Time Before and After Digitalization")
plt.xlabel("Internet Access", fontsize=12, fontweight='bold')
plt.ylabel("Average Application Submission Time")


plt.legend(title="Digitalization", fontsize=11)


In [ ]:

df_analysis = df.groupby(["Treatment", "Treatment_Post"])["e_mutation_cost"].mean().reset_index()


df_analysis["Service_Type"] = df_analysis["Treatment"].map({0: "Manual System", 1: "Digital System"})
df_analysis["Digitalization_Period"] = df_analysis["Treatment_Post"].map({0: "Before Digitalization", 1: "After Digitalization"})


plt.figure(figsize=(8, 5))
ax = sns.barplot(
    x="Service_Type",
    y="e_mutation_cost",
    hue="Digitalization_Period",
    data=df_analysis,
    palette={"Before Digitalization": "blue", "After Digitalization": "orange"},
    capsize=0.1
)

plt.title("Effect of Digital vs Manual System on e-Mutation Cost")
plt.xlabel("Service Type", fontsize=12, fontweight='bold')
plt.ylabel("Average e-Mutation Cost")

plt.legend(title="Digitalization", fontsize=11)



In [ ]:


df_analysis = df.groupby(["Treatment", "Treatment_Post"])["gov_office_visits"].mean().reset_index()


df_analysis["Service_Type"] = df_analysis["Treatment"].map({0: "Manual System", 1: "Digital System"})
df_analysis["Digitalization_Period"] = df_analysis["Treatment_Post"].map({0: "Before Digitalization", 1: "After Digitalization"})

plt.figure(figsize=(8, 5))
ax = sns.barplot(
    x="Service_Type",
    y="gov_office_visits",
    hue="Digitalization_Period",
    data=df_analysis,
    palette={"Before Digitalization": "blue", "After Digitalization": "orange"},
    capsize=0.1
)

# Titles and labels
plt.title("Effect of Digital vs Manual System on Government Office Visits")
plt.xlabel("Service Type", fontsize=12, fontweight='bold')
plt.ylabel("Average Number of Office Visits", fontsize=12, fontweight='bold')

# Add a legend
plt.legend(title="Digitalization", fontsize=11)
